In [8]:
!pip install -q faiss-cpu langchain langchain-community langchain-huggingface sentence-transformers langchain-google-genai python-dotenv langchain_google_genai langchain_huggingface

In [9]:
import os
import getpass

# Set Google API key manually
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

Enter your Google API Key: ··········


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings

# Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Free embedding model (384 dimensions)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("LLM and Embeddings Loaded Successfully")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM and Embeddings Loaded Successfully


In [11]:
vector = embeddings.embed_query("Apple")

print("Vector dimension:", len(vector))
print("First 5 values:", vector[:5])

Vector dimension: 384
First 5 values: [-0.006138487718999386, 0.03101177327334881, 0.06479360908269882, 0.01094149798154831, 0.005267191678285599]


In [12]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

vec_cat = embeddings.embed_query("Cat")
vec_dog = embeddings.embed_query("Dog")
vec_car = embeddings.embed_query("Car")

print("Cat vs Dog:", round(cosine_similarity(vec_cat, vec_dog), 4))
print("Cat vs Car:", round(cosine_similarity(vec_cat, vec_car), 4))

Cat vs Dog: 0.6606
Cat vs Car: 0.4633


In [13]:
from langchain_core.documents import Document

docs = [
    Document(page_content="Piyush's favorite food is Pizza with extra cheese."),
    Document(page_content="The secret password to the lab is 'Blueberry'."),
    Document(page_content="LangChain is a framework for developing applications powered by language models."),
]

print("Documents created.")

Documents created.


In [14]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

print("FAISS index built.")

FAISS index built.


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
Answer based ONLY on the context below:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG Chain Ready.")

RAG Chain Ready.


In [16]:
result = chain.invoke("What is the secret password?")
print(result)

The secret password to the lab is 'Blueberry'.


In [17]:
import faiss
import numpy as np

d = 128      # vector dimension
nb = 10000   # number of vectors

xb = np.random.random((nb, d)).astype('float32')

print("Mock dataset created.")

Mock dataset created.


In [18]:
index_flat = faiss.IndexFlatL2(d)
index_flat.add(xb)

print("Flat Index contains:", index_flat.ntotal, "vectors")

Flat Index contains: 10000 vectors


In [19]:
nlist = 100
quantizer = faiss.IndexFlatL2(d)

index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)

index_ivf.train(xb)
index_ivf.add(xb)

print("IVF Index trained and populated.")

IVF Index trained and populated.


In [20]:
M = 16
index_hnsw = faiss.IndexHNSWFlat(d, M)
index_hnsw.add(xb)

print("HNSW Index created.")

HNSW Index created.


In [21]:
m = 8  # number of sub-vectors

index_pq = faiss.IndexPQ(d, m, 8)
index_pq.train(xb)
index_pq.add(xb)

print("PQ Index created with compression.")

PQ Index created with compression.


In [22]:
chain.invoke("What is Piyush's favorite food?")

"Piyush's favorite food is Pizza with extra cheese."

In [23]:
import nbformat

with open("RAG_Vector_Stores.ipynb", "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove widget metadata entirely
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

with open("clean_notebook.ipynb", "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Cleaned notebook saved as clean_notebook.ipynb")

Cleaned notebook saved as clean_notebook.ipynb
